# Hyperparameter Tuning using HyperDrive


In [1]:
from azureml.core import Workspace, Environment
from azureml.core import Experiment
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core.conda_dependencies import CondaDependencies

from azureml.train.hyperdrive import BayesianParameterSampling, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import normal, uniform, choice

import shutil
import os

## Dataset

In [2]:
data_path = "https://raw.githubusercontent.com/jaircastruita/nd00333-capstone/master/starter_file/data/WA_Fn-UseC_-HR-Employee-Attrition.csv"

ibm_ds = Dataset.Tabular.from_delimited_files(path=data_path)

In [3]:
ws = Workspace.from_config()
experiment_name = 'dnn-experiment'

experiment=Experiment(ws, experiment_name)

## Compute Target

In [4]:
cluster_name = "hd-dnn-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Reusing already created compute target")

except ComputeTargetException:
    print("Creating compute target...")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6", max_nodes=6)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Reusing already created compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-16T20:49:04.909000+00:00', 'errors': None, 'creationTime': '2021-02-16T20:04:52.463488+00:00', 'modifiedTime': '2021-02-16T20:05:08.189712+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [5]:
project_folder = "./dnn-attrition"
os.makedirs(project_folder, exist_ok=True)
shutil.copy("dnn-train.py", project_folder)

'./dnn-attrition/dnn-train.py'

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - torch==1.6.0
  - scikit-learn

Writing conda_dependencies.yml


In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# early_termination_policy = <your policy here>

param_sampling = BayesianParameterSampling({
    "learning_rate": uniform(0.001, 0.1),
    "num_epochs": choice(range(100, 5000)),
    "layer1": choice(range(10, 100)),
    "layer2": choice(range(10, 100)),
})

conda_env = Environment.from_conda_specification(name="PyTorch-1.6-GPU-scikit-learn", file_path="conda_dependencies.yml")
# Specify a GPU base image
conda_env.docker.enabled = True
conda_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

src = ScriptRunConfig(source_directory=project_folder,
                      script="dnn-train.py",
                      arguments=["--num_epochs", 100, "--layer1", 30, "--layer2", 15, "--learning_rate", 0.001,"--output_dir", "./outputs"],
                      compute_target=compute_target,
                      environment=conda_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src, 
                                         hyperparameter_sampling=param_sampling, 
                                         primary_metric_name='best_test_acc', 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=8,
                                         max_concurrent_runs=4)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:80.


In [7]:
# run = experiment.submit(src)
# print(run)

Run(Experiment: dnn-experiment,
Id: dnn-experiment_1613508897_4f5594a0,
Type: azureml.scriptrun,
Status: Starting)


In [8]:
# RunDetails(run).show()
# run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: dnn-experiment_1613508897_4f5594a0
Web View: https://ml.azure.com/experiments/dnn-experiment/runs/dnn-experiment_1613508897_4f5594a0?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-139044/workspaces/quick-starts-ws-139044

Streaming azureml-logs/55_azureml-execution-tvmps_d37cc309f7411fa1b05194646371aa20ceb1588abadaf8b76d63786c39b5b5ea_d.txt

2021-02-16T20:59:33Z Starting output-watcher...
2021-02-16T20:59:34Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-02-16T20:59:35Z Executing 'Copy ACR Details file' on 10.0.0.5
2021-02-16T20:59:35Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_11107b6bcece7d9b98bacf66945d41a0
f22ccc0b8772: Pulling fs layer
3cf8fb62ba5f: Pulling fs layer
e80c964ece6a: Pulling fs layer
a0afc6a19124: Pulling fs layer
18287e0c259e: Pulling fs layer
88c0e0db918c: Pulling fs layer
c2899c05e34e: Pulling f

{'runId': 'dnn-experiment_1613508897_4f5594a0',
 'target': 'hd-dnn-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-16T20:59:31.063298Z',
 'endTimeUtc': '2021-02-16T21:01:48.198402Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'b7cd06a3-93dd-4cf2-991d-8fd53168622d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'dnn-train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--num_epochs',
   '100',
   '--layer1',
   '30',
   '--layer2',
   '15',
   '--learning_rate',
   '0.001',
   '--output_dir',
   './outputs'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'hd-dnn-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'c

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
# assert(hyperdrive_run.get_status == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_548bf18e-5bd9-4f94-b793-df7ab7830507
Web View: https://ml.azure.com/experiments/dnn-experiment/runs/HD_548bf18e-5bd9-4f94-b793-df7ab7830507?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-139044/workspaces/quick-starts-ws-139044

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-16T21:07:59.659587][API][INFO]Experiment created<END>\n""<START>[2021-02-16T21:08:00.257977][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-02-16T21:08:00.6447577Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-16T21:08:00.590802][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_548bf18e-5bd9-4f94-b793-df7ab7830507
Web View: https://ml.azure.com/experiments/dnn-experiment/runs/HD_548bf18e-5bd9-4f94-b793-df7ab7830507?wsid=/subscriptions/cdbe0b4

AssertionError: 

In [11]:
hyperdrive_run.get_status()

'Completed'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)


Run(Experiment: dnn-experiment,
Id: HD_548bf18e-5bd9-4f94-b793-df7ab7830507_7,
Type: azureml.scriptrun,
Status: Completed)


In [18]:
best_run_metrics

{'num_epochs': 4089,
 'learning_rate': 0.07129908970875176,
 'num_layer1': 19,
 'num_layer2': 90,
 'best_test_acc': [0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5005168403824619,
  0.5005168403824619,
  0.5005168403824619,
  0.5045654233784133,
  0.5217934361271427,
  0.5410457403738479,
  0.5516840382461883,
  0.5623223361185288,
  0.5623223361185288,
  0.5623223361185288,
  0.5623223361185288,
  0.5623223361185288,
  0.5623223361185288,
  0.5689120509949178,
  0.5689120509949178,
  0.6266689637350331,
  0.6373072616073735,
  0.6373072616073735,
  0.6373072616073735,
  0.654535274356103,
  0.654535274356103,
  0.654535274356103,
  0.654535274356103,
  0.654535274356103,
  0.654535274356103,
  0.654535274356103,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.661124989232492,
  0.66112498

In [25]:

print('Best Run is:\n  Validation accuracy: {} \n  Learning rate: {} \n Number epochs: {} \n Layer 1: {} \n layer 2: {}'.format(
        best_run_metrics['best_test_acc'][-1],
        best_run_metrics['learning_rate'],
        best_run_metrics['num_epochs'],
        best_run_metrics['num_layer1'],
        best_run_metrics['num_layer2'])
     )

Best Run is:
  Validation accuracy: 0.6996295977259024 
  Learning rate: 0.07129908970875176 
 Number epochs: 4089 
 Layer 1: 19 
 layer 2: 90


In [22]:
#TODO: Save the best model
model = best_run.register_model(model_name='pytorch-attrition', 
                                model_path='outputs/model.pt')
print(model.name, model.id, model.version, sep='\t')

pytorch-attrition	pytorch-attrition:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service